In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
%matplotlib inline

# Data Setup & Functions

In [3]:
from mll_calc.mll_pred import format_XY

In [4]:
def logpdf_calc(row, test_sample, unc):
    y_sim = row[test_sample>0].values.tolist()
    std = row.multiply(unc)[test_sample>0].values.tolist()
    y_mes = test_sample[test_sample>0].values.tolist()
    logpdf_list = stats.norm.logpdf(y_sim, loc=y_mes, scale=std)
    return logpdf_list

In [5]:
def loop_db(XY, pred, unc, lbls, nonlbls):  
    logpdf_df = pd.DataFrame()
    for idx, row in pred.iterrows():
        sim_idx = row['sim_idx']
        pred_idx = row['pred_idx']
        all_lbls = lbls + nonlbls
        
        test_sample = XY.loc[sim_idx].drop(all_lbls)
        train_row = XY.loc[pred_idx].drop(all_lbls)
        
        logpdf = logpdf_calc(train_row, test_sample, unc)
        mll = row['MaxLogLL']
        
        if round(mll, 4) != round(np.sum(logpdf), 4):
            print('mismatch')
            break
        
        logpdf = pd.Series(logpdf, index=train_row[test_sample>0].index, name=sim_idx)
        if logpdf_df.empty:
            logpdf_df = pd.DataFrame(columns = test_sample.index.to_list())
        logpdf_df = logpdf_df.append(logpdf)
    return logpdf_df

### Train and Test DBs

In [6]:
lbls = ['ReactorType', 'CoolingTime', 'Enrichment', 'Burnup', 'OrigenReactor']
nonlbls = ['AvgPowerDensity', 'ModDensity', 'UiWeight']

train_pkl = '~/sims_n_results/simupdates_aug2020/not-scaled_nuc29.pkl'
XY = format_XY(train_pkl)

### LL Calc Results

In [7]:
results = '~/sims_n_results/simupdates_aug2020/train29/'

uncs = [0.05, 0.1, 0.15, 0.2]
job_dirs = ['Job' + str(i) + '_unc' + str(unc) for i, unc in enumerate(uncs)]
dfs = {}
for i, unc_job in enumerate(job_dirs):
    dfs['unc' + str(i)] = pd.read_csv(results + unc_job + '/' + unc_job + '.csv')

for dfname in list(dfs.keys()):
    df = dfs[dfname]
    df['Relative_Burnup_Error'] = df['Burnup_Error'] / df['Burnup']
    df['Relative_Enrichment_Error'] = df['Enrichment_Error'] / df['Enrichment']
    df['Relative_CoolingTime_Error'] = df['CoolingTime_Error'] / df['CoolingTime']

# For ease of only viewing 5% case for now
preds = dfs['unc0']

### LogPDF (wrt nuclide) results

In [8]:
unc = 0.05
logpdf_df = loop_db(XY, preds, unc, lbls, nonlbls)

# Explore Errors

### Burnup

In [9]:
preds['Relative_Burnup_Error'].describe()
# nuc15 results for comparison
#count    9600.000000
#mean        0.002187
#std         0.006690
#min         0.000000
#25%         0.000000
#50%         0.000000
#75%         0.000000
#max         0.064146

count    12000.000000
mean         0.003525
std          0.009948
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          0.149049
Name: Relative_Burnup_Error, dtype: float64

In [10]:
to_print = ['sim_idx', 'pred_idx', 'AvgPowerDensity', 
            'ReactorType', 'pred_ReactorType', 'ReactorType_Score', 
            'Enrichment', 'pred_Enrichment', 'Relative_Enrichment_Error', 
            'Burnup', 'pred_Burnup', 'Relative_Burnup_Error',
            'CoolingTime', 'pred_CoolingTime', 'Relative_CoolingTime_Error',
            'OrigenReactor', 'pred_OrigenReactor', 'MaxLogLL']
preds.loc[preds['Relative_Burnup_Error'] > 0.08, to_print]

,sim_idx,pred_idx,AvgPowerDensity,ReactorType,pred_ReactorType,ReactorType_Score,Enrichment,pred_Enrichment,Relative_Enrichment_Error,Burnup,pred_Burnup,Relative_Burnup_Error,CoolingTime,pred_CoolingTime,Relative_CoolingTime_Error,OrigenReactor,pred_OrigenReactor,MaxLogLL
368,11299,147378,10.0,bwr,bwr,True,1.570,1.870,0.191083,36458.87,39461.41,0.082354,1520.464582,1502.271013,0.011966,ge7x7-0,atrium10x10-9,-122.232230
999,30560,91099,10.0,bwr,bwr,True,2.740,3.100,0.131387,7974.93,8683.04,0.088792,1706.758096,1708.649694,0.001108,ge7x7-0,abb8x8-1,-69.409535
2309,70563,136082,10.0,bwr,bwr,True,1.380,1.610,0.166667,944.34,1030.70,0.091450,7.797336,1.002610,0.871416,abb8x8-1,atrium10x10-9,9.789710
2807,85858,20338,10.0,bwr,bwr,True,2.080,2.030,0.024038,3250.48,2970.68,0.086080,7120.206752,6973.335475,0.020627,abb8x8-1,ge7x7-0,-21.771372
3298,100926,151326,10.0,bwr,bwr,True,4.000,3.280,0.180000,3250.48,2766.00,0.149049,210.929088,242.633977,0.150311,abb8x8-1,atrium10x10-9,-105.995935
5279,161525,45605,10.0,bwr,bwr,True,4.230,4.400,0.040189,5670.60,6261.29,0.104167,109.336506,84.338574,0.228633,atrium10x10-9,ge7x7-0,-62.594131
5293,161885,45965,10.0,bwr,bwr,True,4.230,4.400,0.040189,14697.28,16053.83,0.092299,109.336506,84.338574,0.228633,atrium10x10-9,ge7x7-0,-101.280906
7081,216785,95825,10.0,bwr,bwr,True,2.730,3.100,0.135531,1909.14,2107.24,0.103764,101.346301,98.925867,0.023883,svea64-1,abb8x8-1,-38.692282
9275,283935,173957,25.0,pwr,bwr,False,4.520,5.110,0.130531,20178.59,21847.13,0.082689,1270.348072,1298.076612,0.021828,w17x17,atrium10x10-9,-119.307797
10760,329396,261416,25.0,pwr,pwr,True,4.990,5.390,0.080160,25193.99,27430.12,0.088756,6262.694626,6131.154219,0.021004,bw15x15,ce14x14,-108.454521


### Enrichment

In [11]:
preds['Relative_Enrichment_Error'].describe()
# nuc15 results for comparison
#count    9600.000000
#mean        0.004361
#std         0.019275
#min         0.000000
#25%         0.000000
#50%         0.000000
#75%         0.000000
#max         0.303448

count    12000.000000
mean         0.003752
std          0.014869
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          0.248447
Name: Relative_Enrichment_Error, dtype: float64

In [12]:
preds.loc[preds['Relative_Enrichment_Error'] > 0.12, to_print]

,sim_idx,pred_idx,AvgPowerDensity,ReactorType,pred_ReactorType,ReactorType_Score,Enrichment,pred_Enrichment,Relative_Enrichment_Error,Burnup,pred_Burnup,Relative_Burnup_Error,CoolingTime,pred_CoolingTime,Relative_CoolingTime_Error,OrigenReactor,pred_OrigenReactor,MaxLogLL
368,11299,147378,10.0,bwr,bwr,True,1.57,1.87,0.191083,36458.87,39461.41,0.082354,1520.464582,1502.271013,0.011966,ge7x7-0,atrium10x10-9,-122.232230
531,16338,71656,10.0,bwr,bwr,True,1.57,1.38,0.121019,36458.87,34658.13,0.049391,1385.096982,1340.825727,0.031963,ge7x7-0,abb8x8-1,-119.813632
999,30560,91099,10.0,bwr,bwr,True,2.74,3.10,0.131387,7974.93,8683.04,0.088792,1706.758096,1708.649694,0.001108,ge7x7-0,abb8x8-1,-69.409535
2306,70566,10086,10.0,bwr,bwr,True,1.38,1.57,0.137681,944.34,965.63,0.022545,210.929088,208.534543,0.011352,abb8x8-1,ge7x7-0,-8.041813
2309,70563,136082,10.0,bwr,bwr,True,1.38,1.61,0.166667,944.34,1030.70,0.091450,7.797336,1.002610,0.871416,abb8x8-1,atrium10x10-9,9.789710
3298,100926,151326,10.0,bwr,bwr,True,4.00,3.28,0.180000,3250.48,2766.00,0.149049,210.929088,242.633977,0.150311,abb8x8-1,atrium10x10-9,-105.995935
4280,131105,70625,10.0,bwr,bwr,True,1.61,1.38,0.142857,2132.20,2107.24,0.011706,109.336506,98.925867,0.095216,atrium10x10-9,abb8x8-1,-34.274453
4320,132306,147426,10.0,bwr,bwr,True,1.61,1.87,0.161491,43096.56,43096.56,0.000000,242.633977,242.633977,0.000000,atrium10x10-9,atrium10x10-9,-137.796529
4324,132318,207976,10.0,bwr,bwr,True,1.61,2.01,0.248447,43096.56,45382.93,0.053052,1502.271013,1495.612006,0.004433,atrium10x10-9,svea64-1,-130.158957
4625,141555,131415,10.0,bwr,bwr,True,1.87,1.61,0.139037,9700.27,9304.01,0.040850,1053.708101,1053.708101,0.000000,atrium10x10-9,atrium10x10-9,-87.300122


### Cooling Time

In [13]:
preds['Relative_CoolingTime_Error'].describe()
# nuc15 results for comparison
#count    9600.000000
#mean             inf
#std              NaN
#min         0.000000
#25%         0.004190
#50%         0.013621
#75%         0.047517
#max              inf

count    1.200000e+04
mean              inf
std               NaN
min      0.000000e+00
25%      3.333665e-03
50%      1.209631e-02
75%      2.999413e-02
max               inf
Name: Relative_CoolingTime_Error, dtype: float64

In [14]:
preds.loc[(preds['Relative_CoolingTime_Error'] != np.inf) & (preds['Relative_CoolingTime_Error'] > 2.9), to_print]

,sim_idx,pred_idx,AvgPowerDensity,ReactorType,pred_ReactorType,ReactorType_Score,Enrichment,pred_Enrichment,Relative_Enrichment_Error,Burnup,pred_Burnup,Relative_Burnup_Error,CoolingTime,pred_CoolingTime,Relative_CoolingTime_Error,OrigenReactor,pred_OrigenReactor,MaxLogLL
7,303,304,10.0,bwr,bwr,True,0.500,0.500,0.0,7974.93,7974.93,0.0,6.392266,25.487227,2.987197,ge7x7-0,ge7x7-0,-64.515327
108,3243,3244,20.0,bwr,bwr,True,0.500,0.500,0.0,63534.57,63534.57,0.0,6.392266,25.487227,2.987197,ge7x7-0,ge7x7-0,-108.271418
140,4323,4324,25.0,bwr,bwr,True,0.500,0.500,0.0,29424.92,29424.92,0.0,6.392266,25.487227,2.987197,ge7x7-0,ge7x7-0,-98.302174
556,16983,16984,20.0,bwr,bwr,True,1.570,1.570,0.0,4769.54,4769.54,0.0,6.392266,25.487227,2.987197,ge7x7-0,ge7x7-0,-45.095398
604,18424,18425,20.0,bwr,bwr,True,1.570,1.570,0.0,70500.00,70500.00,0.0,6.392266,25.487227,2.987197,ge7x7-0,ge7x7-0,-112.081377
633,19383,19384,25.0,bwr,bwr,True,1.570,1.570,0.0,22690.61,22690.61,0.0,6.392266,25.487227,2.987197,ge7x7-0,ge7x7-0,-93.441531
650,19923,19924,25.0,bwr,bwr,True,1.570,1.570,0.0,51880.09,51880.09,0.0,6.392266,25.487227,2.987197,ge7x7-0,ge7x7-0,-109.741057
687,21063,21064,10.0,bwr,bwr,True,2.030,2.030,0.0,22690.61,22690.61,0.0,6.392266,25.487227,2.987197,ge7x7-0,ge7x7-0,-94.060324
690,21123,21124,10.0,bwr,bwr,True,2.030,2.030,0.0,29424.92,29424.92,0.0,6.392266,25.487227,2.987197,ge7x7-0,ge7x7-0,-101.480483
754,23043,23044,20.0,bwr,bwr,True,2.030,2.030,0.0,36458.87,36458.87,0.0,6.392266,25.487227,2.987197,ge7x7-0,ge7x7-0,-106.260106


# Max Log LL

In [15]:
print(dfs['unc0']['MaxLogLL'].describe())
print(dfs['unc3']['MaxLogLL'].describe())

count    12000.000000
mean       -71.629155
std         40.399499
min       -168.412372
25%        -99.708077
50%        -84.452120
75%        -54.753846
max         99.078500
Name: MaxLogLL, dtype: float64
count    12000.000000
mean      -108.324892
std         40.808410
min       -163.935972
25%       -137.404639
50%       -121.719013
75%        -92.132707
max         58.775757
Name: MaxLogLL, dtype: float64


# Nuclide Contributions to LL

In [16]:
nuc29 = ['am241', 'am242m', 'am243', 
         'cm242', 'cm244', 
         'cs134', 'cs137', 
         'eu154', 
         'nd143', 'nd144', 'nd145', 'nd146', 'nd148', 'nd150',
         'np237', 
         'pu238', 'pu239', 'pu240', 'pu241', 'pu242',
         'sm147', 'sm149', 'sm150', 'sm151', 'sm152',
         'u234', 'u235', 'u236', 'u238']

In [17]:
logpdf_df.sample(10)

,am241,am242m,am243,cm242,cm244,cs134,cs137,eu154,nd143,nd144,...,pu242,sm147,sm149,sm150,sm151,sm152,u234,u235,u236,u238
174825,1.992885,13.008930,10.530074,18.958014,15.376806,5.971338,-2.055835,5.492767,-2.304358,-2.251700,...,5.660233,-1.313783,0.896601,-0.482756,0.612089,0.324468,-3.775121,-8.712938,-4.433465,-11.684577
365794,-0.036768,9.690976,6.265889,15.636113,10.535789,5.305875,-2.106492,4.268183,-2.192484,-2.239360,...,1.688136,-1.187172,2.219856,-0.673677,1.708684,-0.227789,-3.734048,-6.462161,-3.653306,-11.729070
308793,0.224832,10.196218,7.211633,16.142430,11.263227,4.417425,-2.813668,3.937159,-2.987698,-2.949715,...,3.063775,-1.927003,0.629639,-1.283582,0.048510,-0.472651,-3.751858,-8.736966,-4.993080,-11.679391
157945,3.917349,15.334911,12.934274,20.919202,18.219533,6.051179,-1.450711,6.327551,-1.580853,-1.517835,...,7.435073,-0.448276,1.405892,0.314463,1.240695,1.085396,-3.784277,-8.283169,-3.919161,-11.704229
94869,-2.746412,-4.296142,-0.645903,1.065061,1.267323,-1.471454,-4.479232,0.080371,-4.251424,-4.521489,...,-3.009556,-3.430280,-0.268251,-3.074015,-0.015369,-2.256014,-3.514594,-7.446272,-5.956237,-11.694138
27143,-1.415145,6.617875,3.209377,10.240970,6.426845,1.901941,-3.262888,2.418524,-3.258819,-3.345549,...,-0.315632,-2.100644,1.496256,-1.812984,0.598894,-1.217561,-3.689024,-7.479262,-5.020755,-11.713296
230151,-4.510632,2.288056,-4.297038,8.237342,-3.589208,0.554842,-5.397323,-0.699410,-4.724156,-6.055913,...,-5.347372,-3.786569,1.331529,-4.224176,-0.306619,-3.044786,-3.165914,-4.972461,-6.538252,-11.643417
181051,-4.150931,2.264076,-2.504391,8.201394,-1.015282,0.038917,-5.076380,-0.530581,-4.896749,-5.427598,...,-4.237298,-3.768928,0.939923,-3.822830,-0.392296,-2.825514,-3.365661,-7.442501,-6.678156,-11.661404
162890,-4.763709,1.051528,-4.347998,6.997286,-3.760243,1.388917,-5.298842,-0.743110,-4.862243,-6.047689,...,-5.315088,-3.909671,1.374871,-4.157993,-0.398653,-2.965578,-3.405926,-5.941289,-6.596000,-11.640451
171495,0.449370,10.827493,9.267704,-14.577415,13.483675,1.232100,-2.527150,4.153249,-2.556964,-2.508433,...,4.657375,-1.882239,0.949270,-0.810239,0.366223,0.002993,-3.768199,-8.697569,-4.621844,-11.684261


In [18]:
for nuc in nuc29:
    desc = logpdf_df[nuc].describe()
    print(desc[['mean', 'std', 'min', 'max']])

mean    -2.696207
std      2.158754
min    -50.481452
max     10.460768
Name: am241, dtype: float64
mean     4.010090
std      3.181927
min    -11.232377
max     17.495760
Name: am242m, dtype: float64
mean    -0.553649
std      4.107314
min     -7.402710
max     15.239458
Name: am243, dtype: float64
mean     6.936771
std      5.606717
min    -98.357667
max     23.444578
Name: cm242, dtype: float64
mean     1.284471
std      5.431075
min    -13.565342
max     21.322739
Name: cm244, dtype: float64
mean     0.379066
std      2.702140
min    -59.345434
max     10.808199
Name: cs134, dtype: float64
mean   -4.286094
std     1.037328
min    -8.998465
max    -0.458919
Name: cs137, dtype: float64
mean    0.659762
std     1.909074
min    -5.813059
max     7.917447
Name: eu154, dtype: float64
mean   -3.967636
std     0.800308
min    -9.333652
max    -0.667267
Name: nd143, dtype: float64
mean    -4.490245
std      1.235986
min    -18.161401
max      1.649567
Name: nd144, dtype: float64
mean   -3.8

In [35]:
arr = []
for nuc in nuc29:
    desc = logpdf_df[nuc].describe()
    arr.append((desc['mean'], desc['std'], desc['min'], desc['max']))
logpdf_nuc = pd.DataFrame(arr, index=nuc29, columns=['mean', 'std', 'min', 'max'])

In [39]:
logpdf_nuc

,mean,std,min,max
am241,-2.696207,2.158754,-50.481452,10.460768
am242m,4.010090,3.181927,-11.232377,17.495760
am243,-0.553649,4.107314,-7.402710,15.239458
cm242,6.936771,5.606717,-98.357667,23.444578
cm244,1.284471,5.431075,-13.565342,21.322739
cs134,0.379066,2.702140,-59.345434,10.808199
cs137,-4.286094,1.037328,-8.998465,-0.458919
eu154,0.659762,1.909074,-5.813059,7.917447
nd143,-3.967636,0.800308,-9.333652,-0.667267
nd144,-4.490245,1.235986,-18.161401,1.649567


In [41]:
logpdf_nuc['mean'].nlargest(7)

cm242     6.936771
am242m    4.010090
cm244     1.284471
sm149     1.175245
eu154     0.659762
cs134     0.379066
sm151     0.091126
Name: mean, dtype: float64

In [42]:
logpdf_nuc['max'].nlargest(7)

cm242     23.444578
cm244     21.322739
am242m    17.495760
am243     15.239458
cs134     10.808199
am241     10.460768
pu242      9.318205
Name: max, dtype: float64